# Setup

In [1]:
using DifferentialEquations, DiffEqBayes, Plots, Turing

# Three node network

Based on lorenz example. Writing out the system of equations individually instead of looping through each node.

In [2]:
phi(a::Number) = (exp(2*a)-1)/(exp(2*a)+1)

phi (generic function with 1 method)

In [ ]:
function additive_noise!(du,u,p,t)
    s,g,W,I,σ = p
    for i = 1:length(u)
        du[i] = du[i]+randn(1)*σ
    end
end

function network_model!(du,u,p,t)
    s,g,W,I,σ = p
    for i=1:length(u)
        du[i] = -u[i] + s*phi(u[i]) + g*(W*u) + I(t)
    end
end

In [ ]:
num_nodes = 3
u0 = repeat([0.0], num_nodes)
Tmax = 100.0
tspan = (0.0,Tmax)
dt = 0.5
s = 0.3
g = 0.7
W = [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0]
I
σ = 0.01

#Task input has to be a continuous function for I(t) to work as a parameter in the DEP definition
#So you'll prob have to use interpolation 

p = [s,g,W,I,σ]

prob_sde = SDEProblem(network_model!,additive_noise!,u0,tspan,p)


3×3 Array{Float64,2}:
 0.0  0.2  0.0
 0.4  0.0  0.0
 0.0  0.3  0.0